# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
from pymongo import MongoClient
import re
import pandas as pd
from pymongo import ASCENDING, DESCENDING
client = MongoClient(f"mongodb://localhost/companies")
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
searchre = re.compile("^Babelgum.*")
cursor = db.companies.find({"name":searchre})
for e in cursor:
    print(e["name"])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [3]:
query={"number_of_employees":{'$gte': 5000}}
data=list(db.companies.find(query,{"name":1, "number_of_employees":1,"_id":0}).sort('number_of_employees', ASCENDING).limit(20))
df = pd.DataFrame(data)
display(df)

,name,number_of_employees
0,Spotify,5000
1,PCH International,5000
2,McAfee,5000
3,JDS Uniphase Corporation,5000
4,Air Astana Airlines,5000
5,United Internet,5000
6,Vcare Call Center,5000
7,Nintendo,5080
8,Hexaware Technologies,5200
9,Facebook,5299


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [4]:
query = { "founded_year": { '$gte': 2000, '$lt': 2006 }}
data= db.companies.find(query,{"name":1, "founded_year":1,"_id":0})
df = pd.DataFrame(data)
display(df.head())

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Omnidrive,2005
3,Digg,2004
4,StumbleUpon,2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [5]:
query = {'$and': [{"founded_year":{'$lt':2010}}, {"ipo.valuation_amount":{'$gte':100000000}}]}
data= db.companies.find(query,{"name":1, "ipo":1,"_id":0})
#pd.json_normalize(): normalize semi-structured JSON data into a flat table.
df = pd.json_normalize(data)
display(df.head())

,name,ipo.valuation_amount,ipo.valuation_currency_code,ipo.pub_year,ipo.pub_month,ipo.pub_day,ipo.stock_symbol
0,Facebook,104000000000,USD,2012.0,5.0,18.0,NASDAQ:FB
1,Yelp,1300000000,USD,2012.0,3.0,2.0,NYSE:YELP
2,LinkedIn,9310000000,USD,2011.0,7.0,20.0,NYSE:LNKD
3,Amazon,100000000000,USD,1997.0,5.0,NaN,NASDAQ:AMZN
4,Brightcove,290000000,USD,2012.0,2.0,17.0,NASDAQ:BCOV


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [6]:
query = {'$and': [{"number_of_employees":{'$lt':1000}}, {"founded_year":{'$lt':2005}}]}
data= db.companies.find(query,{"name":1,"_id":0}).sort('number_of_employees', DESCENDING).limit(10)
df = pd.DataFrame(data)
display(df)

,name
0,Infinera Corporation
1,NorthPoint Communications Group
2,888 Holdings
3,Forrester Research
4,SonicWALL
5,Webmetrics
6,Cornerstone OnDemand
7,Yelp
8,MySpace
9,ZoomInfo


### 6. All the companies that don't include the `partners` field.

In [7]:
query = {'partners':{ '$exists':False } } 
data= db.companies.find(query,{"name":1,"_id":0})
df = pd.DataFrame(data)
df.shape

(0, 0)

### 7. All the companies that have a null type of value on the `category_code` field.

In [8]:
query = {'category_code':{ '$type':'null' } } 
data = db.companies.find(query,{"name":1,"_id":0})
df = pd.DataFrame(data)
display(df.head())

,name
0,Collective
1,KoolIM
2,Snimmer
3,Level9 Media
4,VidKing


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [9]:
query = { "number_of_employees": { '$gte': 100, '$lt': 1000 }}
data= db.companies.find(query,{"name":1, "number_of_employees":1,"_id":0})
df = pd.DataFrame(data)
display(df.head())

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [10]:
query = {"ipo.valuation_amount":{ '$type':'int' }}
data= db.companies.find(query,{"name":1, "ipo.valuation_amount":1,"_id":0}).sort('ipo.valuation_amount', DESCENDING)
df = pd.json_normalize(data)
display(df.head())

,name,ipo.valuation_amount
0,Yelp,1300000000
1,Chegg,1100000000
2,Chegg,1100000000
3,RPX Corporation,1096000000
4,Higher One,1060000000


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [11]:
query = {'number_of_employees':{ '$exists':True } } 
data= db.companies.find(query,{"name":1, "number_of_employees":1,"_id":0}).sort('number_of_employees', DESCENDING).limit(10)
df = pd.DataFrame(data)
display(df)

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [12]:
query = {'founded_month': {'$gte': 6, '$lte': 12}}  
data= db.companies.find(query,{"name":1, "founded_month":1,"_id":0}).limit(1000)
df = pd.DataFrame(data)
display(df.head())

,name,founded_month
0,Wetpaint,10
1,Zoho,9
2,Postini,6
3,Omnidrive,11
4,Geni,6


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [13]:
query = {'$and': [{'founded_year': {'$lt': 2000}}, {"acquisition.price_amount":{'$gt':1000000 }} ]}  
data= db.companies.find(query,{"name":1,'founded_year':1,'acquisition.price_amount':1 ,"_id":0})
df = pd.json_normalize(data)
display(df.head())

,name,founded_year,acquisition.price_amount
0,SideStep,1999,180000000
1,Recipezaar,1999,25000000
2,Cyworld,1999,7140000
3,PayPal,1998,1500000000
4,Snapfish,1999,300000000


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [14]:
query = {'acquisition.acquired_year': {'$gt': 2010}} 
data= db.companies.find(query,{"name":1,'acquisition':1,"_id":0}).sort('acquisition.price_amount',DESCENDING)
df = pd.json_normalize(data)
display(df.head())

,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,Goodrich Corporation,1.840000e+10,USD,None,http://www.masshightech.com/stories/2011/09/19...,UTC shells out $18.4 billion for Goodrich,2011,9.0,22.0,United Technologies,united-technologies
1,LSI,6.600000e+09,USD,cash,http://dealbook.nytimes.com/2013/12/16/avago-t...,Avago to Buy LSI for $6.6 Billion,2013,12.0,16.0,Avago Technologies,avago-technologies
2,National Semiconductor,6.500000e+09,USD,cash,"http://www.national.com/news/item/0,1735,1522,...",TI to acquire National Semiconductor Complemen...,2011,4.0,4.0,Texas Instruments,texas-instruments
3,Ariba,4.300000e+09,USD,None,http://techcrunch.com/2012/05/22/sap-to-acquir...,SAP To Acquire Ariba For $4.3 Billion,2012,5.0,22.0,SAP,sap
4,NetLogic Microsystems,3.700000e+09,USD,cash_and_stock,http://techcrunch.com/2011/09/12/broadcom-buys...,Broadcom Buys Chipmaking Technology Company Ne...,2011,9.0,12.0,Broadcom,broadcom


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [15]:
query = {'founded_year': {'$exists': True}} 
data= db.companies.find(query,{"name":1,'founded_year':1,"_id":0}).sort('founded_year', ASCENDING)
#No puedo formar un dataframe con esto xq se come toda la RAM?
#Cómo se solucionaría esto? Puedo coger una seleccion más pequeña pero
#Y si me hace falta todo?

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [16]:
query =  {'founded_day': {'$lte':7}} 
data= db.companies.find(query,{"name":1,'founded_day':1,'acquisition.price_amount':1,"_id":0}).sort('acquisition.price_amount', DESCENDING).limit(10)
df = pd.json_normalize(data)
display(df)

,name,founded_day,acquisition.price_amount
0,PayPal,1,1500000000
1,Zappos,1,1200000000
2,Alibaba,1,1000000000
3,Postini,2,625000000
4,Danger,1,500000000
5,Clearwell Systems,6,410000000
6,PrimeSense,1,345000000
7,Amobee,1,321000000
8,BlueLithium,1,300000000


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [17]:
query =  {'$and': [{'category_code': 'web'},{'number_of_employees': {'$gt':4000}}]}
data= db.companies.find(query,{"name":1,'number_of_employees':1,'category_code':1,"_id":0}).sort('number_of_employees', ASCENDING)
df = pd.DataFrame(data)
display(df.head())

,name,category_code,number_of_employees
0,Expedia,web,4400
1,AOL,web,8000
2,Webkinz,web,8657
3,Rakuten,web,10000
4,Los Angeles Times Media Group,web,10000


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [18]:
query =  {'$and': [{'acquisition.price_currency_code': 'EUR'},{'acquisition.price_amount': {'$gt':10000000}}]}
data= db.companies.find(query,{"name":1,'acquisition.price_amount':1,'acquisition.price_currency_code':1,"_id":0})
df = pd.json_normalize(data)
display(df.head())

,name,acquisition.price_amount,acquisition.price_currency_code
0,Apertio,140000000,EUR
1,Greenfield Online,40000000,EUR
2,Webedia,70000000,EUR
3,Wayfinder,24000000,EUR
4,Tuenti Technologies,70000000,EUR


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [19]:
query = {'acquisition.acquired_month': {'$gte': 1, '$lte': 4}}  
data= db.companies.find(query,{"name":1,'acquisition':1,"_id":0})
df = pd.json_normalize(data)
display(df.head())

,name,acquisition.price_amount,acquisition.price_currency_code,acquisition.term_code,acquisition.source_url,acquisition.source_description,acquisition.acquired_year,acquisition.acquired_month,acquisition.acquired_day,acquisition.acquiring_company.name,acquisition.acquiring_company.permalink
0,Veoh,NaN,USD,None,http://techcrunch.com/2010/04/07/qlipso-veoh/,Qlipso's CEO Explains Why He Bought Veoh,2010,4,7.0,Qlipso,qlipso
1,Kyte,NaN,USD,None,http://techcrunch.com/2011/01/31/exclusive-kit...,"KIT digital Acquires KickApps, Kewego AND Kyte...",2011,1,31.0,KIT digital,kit-digital
2,Jingle Networks,62500000.0,USD,cash_and_stock,http://www.masshightech.com/stories/2011/04/11...,Jingle Networks bought for up to $62.5M,2011,4,11.0,Marchex,marchex
3,NetRatings,327000000.0,USD,cash,http://login.vnuemedia.com/hr/login/login_subs...,Nielsen buys rest of NetRatings,2007,2,NaN,Nielsen,nielsen
4,blogTV,NaN,USD,None,http://techcrunch.com/2013/03/13/younow-buys-b...,Live Social Video Network YouNow Acquires Stre...,2013,3,13.0,YouNow,younow


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [20]:
query = {'$and': [ {'acquisition.acquired_year': {'$gte': 2011}}, {'founded_year': {'$gte': 2000, '$lte': 2010}} ]} 
data= db.companies.find(query,{"name":1,'acquisition.acquired_year':1,'founded_year':1 ,"_id":0})
df = pd.json_normalize(data)
display(df.head())

,name,founded_year,acquisition.acquired_year
0,Geni,2006,2012
1,Digg,2004,2012
2,Kyte,2006,2011
3,Jingle Networks,2005,2011
4,blogTV,2006,2013


### 20. All the companies that have been 'deadpooled' after the third year.

In [21]:
data = db.companies.aggregate( [ { '$project': { 'name': 1, 'dateDifference': { '$subtract': [ "$deadpooled_year", "$founded_year" ] } } } ] )
df = pd.json_normalize(data)
df=df.dropna()
df=df[df["dateDifference"]>=3]
display(df.head())

,_id,name,dateDifference
3,52cdef7c4bab8bd675297d8f,Omnidrive,3.0
18,52cdef7c4bab8bd675297da0,Babelgum,6.0
32,52cdef7c4bab8bd675297dae,Thoof,7.0
40,52cdef7c4bab8bd675297db6,Wesabe,5.0
43,52cdef7c4bab8bd675297db7,Jangl SMS,3.0
